For our final project, we decided to create a recommender system that will take any given stock in the S&P 500 and output stocks that are similar to it. The problem we are trying to solve is the complexity of the stock market. It can be difficult to choose which stocks to invest in and to understand which stocks are similar to one another without having to look at complicated financials for a company. We decided to use important financial data to choose stocks that were similar to eachother and then output these choices to the user, making their lives much easier. The user of our function can input one of their favorite stocks that they own in their portfolio and then our function's output will show the user 20 stocks that are very similar. The user could then consider investing in these stocks that were outputted or just use it as informative information. We essentially are taking something like Netflix's use of recommender systems for movies but applying it to the stock market. There really aren't too many resources available for a user to get stocks recommended to them based on similarity, which is part of the reason why we wanted to do a project like this. Working hands-on with creating a recommender system and developing a scoring function gave us insight into how other companies may go about creating recommender systems. Overall, it was a fun and valuable project in which we were able to learn a lot from. 

In [4]:
# mount my google drive to this notebook so I can access my files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# import statements that we will need various operations
import pandas as pd
import numpy as np

In [6]:
# read the csv files and turn them into a dataframe

# website for this data table: https://datahub.io/core/s-and-p-500-companies-financials
stock_csv = '/content/drive/My Drive/DS 397/stock_company_data.csv'
stock_df = pd.read_csv(stock_csv)

# website for this data table: https://finasko.com/sp-500-companies-weightage/ 
weight_csv = '/content/drive/My Drive/DS 397/stock_weights.csv'
weight_df = pd.read_csv(weight_csv)

In [7]:
# drops unneeded columns
weight_df = weight_df.drop("Rank", axis = 1)
weight_df = weight_df.drop("Unnamed: 4", axis = 1)
weight_df = weight_df.drop("Last Update: 24 March 2022", axis = 1)

In [8]:
# drops unneeded columns
stock_df = stock_df.drop("Price", axis = 1)
stock_df = stock_df.drop("52 Week Low", axis = 1)
stock_df = stock_df.drop("52 Week High", axis = 1)
stock_df = stock_df.drop("EBITDA", axis = 1)
stock_df = stock_df.drop("Price/Sales", axis = 1)
stock_df = stock_df.drop("SEC Filings", axis = 1)

In [9]:
# merges the two data tables based on the symbol column using a left join
# this means that all of the rows in the stock_df dataframe will be there
# and any rows in the weight_df that dont match with the stock_df will be null values
merged_df = stock_df.merge(weight_df, on = 'Symbol', how = 'left')

# drops the company column since we already have the name column which is exactly the same
merged_df = merged_df.drop("Company", axis = 1)

# renames the name column to Company Name to be more specific
merged_df.rename(columns = {"Name": "Company Name"}, inplace = True)

In [10]:
# creates a score column of all zeros
merged_df['Score'] = 0

In [20]:
# function to output the dataframe of similar ticker symbols
def test(ticker, stock_df):
  # score function to keep track of each individual ticker's similarity score
  score = 0

  # gets each value from each column of the user specified ticker
  try:
    ticker_sector = stock_df[(stock_df["Symbol"] == ticker)]["Sector"].values[0]
  except IndexError:
    return "Sorry, this is not a valid ticker symbol. Please try again."
  ticker_pe = stock_df[(stock_df["Symbol"] == ticker)]["Price/Earnings"].values[0]
  ticker_div = stock_df[(stock_df["Symbol"] == ticker)]["Dividend Yield"].values[0]
  ticker_earn = stock_df[(stock_df["Symbol"] == ticker)]["Earnings/Share"].values[0]
  ticker_marcap = stock_df[(stock_df["Symbol"] == ticker)]["Market Cap"].values[0]
  ticker_pb = stock_df[(stock_df["Symbol"] == ticker)]["Price/Book"].values[0]
  ticker_weight = stock_df[(stock_df["Symbol"] == ticker)]["Weight"].values[0]

  # finds the standard deviation of each specified column 
  div_std = stock_df["Dividend Yield"].std() / 2
  pe_std = stock_df["Price/Earnings"].std() / 3
  earn_std = stock_df["Earnings/Share"].std()
  marcap_std = stock_df["Market Cap"].std()
  pb_std = stock_df["Price/Book"].std() / 2
  
  # saves the user input ticker symbol as a row and creates a new dataframe without that ticker symbol
  # we do this because there is no point of checking the similarity since they are the same ticker
  # we will add the ticker_row to the final dataframe at index 0 so the user can see the ticker they chose
  # and then all the similar tickers  
  ticker_row = stock_df[stock_df["Symbol"] == ticker]
  new_df = stock_df[stock_df["Symbol"] != ticker]
  
  # for loop to iterate through each ticker symbol in the dataframe
  for company in new_df["Symbol"]:

    # if the company is from the same sector we give this a lot of weight
    if new_df[(new_df["Symbol"] == company)]["Sector"].values[0] == ticker_sector:
      score += 0.8
    # the next 7 if statements will add a certain value to the score variable depending on
    # if the ticker symbol's values for each column falls within 1 standard deviation or so of the user input's ticker symbol value for that
    # specific column
    if (ticker_div - div_std) <= new_df[(new_df["Symbol"] == company)]["Dividend Yield"].values[0] <= (ticker_div + div_std):
      score += 0.7
    if (ticker_pe - pe_std) <= new_df[(new_df["Symbol"] == company)]["Price/Earnings"].values[0] <= (ticker_pe + pe_std):
      score += 0.4
    if (ticker_earn - earn_std) <= new_df[(new_df["Symbol"] == company)]["Earnings/Share"].values[0] <= (ticker_earn + earn_std):
      score += 0.4
    if (ticker_marcap - marcap_std) <= new_df[(new_df["Symbol"] == company)]["Market Cap"].values[0] <= (ticker_marcap + marcap_std):
      score += 0.6
    if (ticker_pb - pb_std) <= new_df[(new_df["Symbol"] == company)]["Price/Book"].values[0] <= (ticker_pb + pb_std):
      score += 0.2
    if (ticker_weight - (ticker_weight/1.2)) <= new_df[(new_df["Symbol"] == company)]["Weight"].values[0] <= (ticker_weight + (ticker_weight/1.2)):
      score += 0.7
    
    # once we iterate through each column for the ticker symbol and finalize the similarity score, we replace the zero value
    # in the score column with the new similarity score
    new_df.loc[new_df['Symbol'] == company, 'Score'] = score

    # we then put the score back to zero for the next ticker symbol
    score = 0


  # once we iterate through all of the ticker symbols, we take the 20 most similar ticker symbols based on the similarity scores
  score_df = new_df.nlargest(20, ['Score']).reset_index(drop=True)
  
  # this chunk of code allows us to get a random selection of tickers with the same lowest score
  # if the lowest score out of the 20 outputted tickers is 2.5, and there are 5 tickers with score 2.5 in the outputted
  # dataframe but really there are 20 tickers with score 2.5 in total, then only the first 5 tickers in alphabetical order will
  # be returned everytime. To fix this, we would randomly shuffle these 20 tickers and then choose 5 of them, this way
  # each ticker with score 2.5 has a chance of being included in the final dataframe, instead of only the first 5 in alphabetical order
  min_score = min(score_df["Score"])
  min_score_df = new_df[new_df["Score"] == min_score]
  min_score_df = min_score_df.sample(frac=1)
  score_df = score_df[score_df['Score'] > min_score]
  num_tickers = len(score_df)
  min_score_samp = min_score_df.head(20-num_tickers)

  # we append the information of the user input ticker symbol to the top of the df
  score_df = pd.concat([ticker_row, score_df, min_score_samp]).reset_index(drop = True)

  # we then return the symbol, name, and sector column as a dataframe for the user
  # this allows the user to see which stocks are most similar to the one they chose
  return score_df[['Symbol', "Company Name", "Sector", "Score"]]

In [14]:
# outputs the 20 most similar stocks to Facebook
# the stock at index 0 is the one that the user chose
facebook_df = test("FB", merged_df)
facebook_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Score
0,FB,"Facebook, Inc.",Information Technology,0.0
1,MA,Mastercard Inc.,Information Technology,3.2
2,V,Visa Inc.,Information Technology,3.2
3,ADBE,Adobe Systems Inc,Information Technology,2.8
4,AMD,Advanced Micro Devices Inc,Information Technology,2.8
5,GOOGL,Alphabet Inc Class A,Information Technology,2.8
6,GOOG,Alphabet Inc Class C,Information Technology,2.8
7,MU,Micron Technology,Information Technology,2.8
8,NFLX,Netflix Inc.,Information Technology,2.8
9,NVDA,Nvidia Corporation,Information Technology,2.8


In [ ]:
# outputs the 20 most similar stocks to Walmart
# the stock at index 0 is the one that the user chose
walmart_df = test("WMT", merged_df)
walmart_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Score
0,WMT,Wal-Mart Stores,Consumer Staples,0.0
1,CVS,CVS Health,Consumer Staples,3.2
2,KR,Kroger Co.,Consumer Staples,3.2
3,MDLZ,Mondelez International,Consumer Staples,3.2
4,SYY,Sysco Corp.,Consumer Staples,3.2
5,WBA,Walgreens Boots Alliance,Consumer Staples,3.2
6,BAC,Bank of America Corp,Financials,3.0
7,CL,Colgate-Palmolive,Consumer Staples,3.0
8,WFC,Wells Fargo,Financials,3.0
9,HD,Home Depot,Consumer Discretionary,2.8


In [ ]:
# outputs the 20 most similar stocks to Nike
# the stock at index 0 is the one that the user chose
nike_df = test("NKE", merged_df)
nike_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Score
0,NKE,Nike,Consumer Discretionary,0.0
1,CMCSA,Comcast Corp.,Consumer Discretionary,3.8
2,DG,Dollar General,Consumer Discretionary,3.8
3,EXPE,Expedia Inc.,Consumer Discretionary,3.8
4,HLT,Hilton Worldwide Holdings Inc,Consumer Discretionary,3.8
5,LOW,Lowe's Cos.,Consumer Discretionary,3.8
6,MAR,Marriott Int'l.,Consumer Discretionary,3.8
7,ROST,Ross Stores,Consumer Discretionary,3.8
8,DIS,The Walt Disney Company,Consumer Discretionary,3.8
9,TJX,TJX Companies Inc.,Consumer Discretionary,3.8


In [ ]:
# outputs the 20 most similar stocks to Proctor & Gamble
# the stock at index 0 is the one that the user chose
pg_df = test("PG", merged_df)
pg_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Score
0,PG,Procter & Gamble,Consumer Staples,0.0
1,MO,Altria Group Inc,Consumer Staples,3.8
2,KO,Coca-Cola Company (The),Consumer Staples,3.8
3,PEP,PepsiCo Inc.,Consumer Staples,3.6
4,CVS,CVS Health,Consumer Staples,3.2
5,AMGN,Amgen Inc,Health Care,3.0
6,CVX,Chevron Corp.,Energy,3.0
7,CSCO,Cisco Systems,Information Technology,3.0
8,INTC,Intel Corp.,Information Technology,3.0
9,IBM,International Business Machines,Information Technology,3.0


**Alternative Method**

For an alternative method in which we give more emphasis on recommending large cap stocks with large cap stocks, and vice versa, we will change the scoring for our function to have weight and market cap as the most two important features. We will change the weight scoring from 0.7 to 0.8 and market cap scoring from 0.6 to 0.7. Additionally, we want to increase the weight of Earnings/Share from 0.4 to 0.7 since we want to place greater emphasis on growth stocks that have been profitable compared to stocks that are stagnant that don't have much room for growth. We will place less of an emphasis on Sector in this function and drop its scoring from 0.8 to 0.2. We are doing this because we are more concerned with matching large growth stocks with other large growth stocks and small stocks with other small stocks, so Sector wouldn't matter. We also want to place less of an emphasis on Dividend Yield because stocks that are growing tend to keep all profits for themselves so that they can reinvest in their company and help it to keep growing; so we will change this from 0.7 to 0.2. Price/Earnings will be decreased slightly from 0.4 to 0.3 since price can vary between stocks and won't really help too much in our ultimate goal of this recommender system. Price/Book will be kept the same at 0.2. Essentially, we want this recommender system to focus mainly on growth stocks that have potential to continue to grow at a rapid pace. As previously mentioned, we also want to ensure that stocks with large influence on the S&P 500 index are also recommended with eachother. We provided two examples below to show the output of our new alternative recommender system.

In [42]:
# new function to output the dataframe of similar ticker symbols using different weights than the original
def test_new(ticker, stock_df):
  # score function to keep track of each individual ticker's similarity score
  score = 0

  # gets each value from each column of the user specified ticker
  try:
    ticker_sector = stock_df[(stock_df["Symbol"] == ticker)]["Sector"].values[0]
  except IndexError:
    return "Sorry, this is not a valid ticker symbol. Please try again."
  ticker_pe = stock_df[(stock_df["Symbol"] == ticker)]["Price/Earnings"].values[0]
  ticker_div = stock_df[(stock_df["Symbol"] == ticker)]["Dividend Yield"].values[0]
  ticker_earn = stock_df[(stock_df["Symbol"] == ticker)]["Earnings/Share"].values[0]
  ticker_marcap = stock_df[(stock_df["Symbol"] == ticker)]["Market Cap"].values[0]
  ticker_pb = stock_df[(stock_df["Symbol"] == ticker)]["Price/Book"].values[0]
  ticker_weight = stock_df[(stock_df["Symbol"] == ticker)]["Weight"].values[0]

  # finds the standard deviation of each specified column 
  div_std = stock_df["Dividend Yield"].std() / 2
  pe_std = stock_df["Price/Earnings"].std() / 3
  earn_std = stock_df["Earnings/Share"].std()
  marcap_std = stock_df["Market Cap"].std() 
  pb_std = stock_df["Price/Book"].std() / 2
  
  # saves the user input ticker symbol as a row and creates a new dataframe without that ticker symbol
  # we do this because there is no point of checking the similarity since they are the same ticker
  # we will add the ticker_row to the final dataframe at index 0 so the user can see the ticker they chose
  # and then all the similar tickers  
  ticker_row = stock_df[stock_df["Symbol"] == ticker]
  new_df = stock_df[stock_df["Symbol"] != ticker]
  
  # for loop to iterate through each ticker symbol in the dataframe
  for company in new_df["Symbol"]:

    # if the company is from the same sector we give this a lot of weight
    if new_df[(new_df["Symbol"] == company)]["Sector"].values[0] == ticker_sector:
      score += 0.2
    # the next 7 if statements will add a certain value to the score variable depending on
    # if the ticker symbol's values for each column falls within 1 standard deviation or so of the user input's ticker symbol value for that
    # specific column
    if (ticker_div - div_std) <= new_df[(new_df["Symbol"] == company)]["Dividend Yield"].values[0] <= (ticker_div + div_std):
      score += 0.2
    if (ticker_pe - pe_std) <= new_df[(new_df["Symbol"] == company)]["Price/Earnings"].values[0] <= (ticker_pe + pe_std):
      score += 0.3
    if (ticker_earn - earn_std) <= new_df[(new_df["Symbol"] == company)]["Earnings/Share"].values[0] <= (ticker_earn + earn_std):
      score += 0.7
    if (ticker_marcap - marcap_std) <= new_df[(new_df["Symbol"] == company)]["Market Cap"].values[0] <= (ticker_marcap + marcap_std):
      score += 0.7
    if (ticker_pb - pb_std) <= new_df[(new_df["Symbol"] == company)]["Price/Book"].values[0] <= (ticker_pb + pb_std):
      score += 0.2
    if (ticker_weight - (ticker_weight/1.2)) <= new_df[(new_df["Symbol"] == company)]["Weight"].values[0] <= (ticker_weight + (ticker_weight/1.2)):
      score += 0.8
    
    # once we iterate through each column for the ticker symbol and finalize the similarity score, we replace the zero value
    # in the score column with the new similarity score
    new_df.loc[new_df['Symbol'] == company, 'Score'] = score

    # we then put the score back to zero for the next ticker symbol
    score = 0


  # once we iterate through all of the ticker symbols, we take the 20 most similar ticker symbols based on the similarity scores
  score_df = new_df.nlargest(20, ['Score']).reset_index(drop=True)
  
  # this chunk of code allows us to get a random selection of tickers with the same lowest score
  # if the lowest score out of the 20 outputted tickers is 2.5, and there are 5 tickers with score 2.5 in the outputted
  # dataframe but really there are 20 tickers with score 2.5 in total, then only the first 5 tickers in alphabetical order will
  # be returned everytime. To fix this, we would randomly shuffle these 20 tickers and then choose 5 of them, this way
  # each ticker with score 2.5 has a chance of being included in the final dataframe, instead of only the first 5 in alphabetical order
  min_score = min(score_df["Score"])
  min_score_df = new_df[new_df["Score"] == min_score]
  min_score_df = min_score_df.sample(frac=1)
  score_df = score_df[score_df['Score'] > min_score]
  num_tickers = len(score_df)
  min_score_samp = min_score_df.head(20-num_tickers)

  # we append the information of the user input ticker symbol to the top of the df
  score_df = pd.concat([ticker_row, score_df, min_score_samp]).reset_index(drop = True)

  # we then return the symbol, name, and sector column as a dataframe for the user
  # this allows the user to see which stocks are most similar to the one they chose
  return score_df[['Symbol', "Company Name", "Sector", "Weight", "Score"]]

As you can see below, our alternative method recommends other large weighted high cap stocks when given Apple. Stocks like Google and Amazon are recommended with it which is what we wanted for our recommender system. Though, there are also other low weight stocks in the mix that made it due to them having similar Earnings/Share. When comparing this to our original function, we can see that there are several new stocks from various Sectors on our alternative approach.

In [47]:
Apple_df_alt = test_new("AAPL", merged_df)
Apple_df_alt

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Weight,Score
0,AAPL,Apple Inc.,Information Technology,6.864361,0.0
1,FB,"Facebook, Inc.",Information Technology,1.312032,2.2
2,UNH,United Health Group Inc.,Health Care,1.248235,2.2
3,BRK.B,Berkshire Hathaway,Financials,1.680761,2.0
4,GOOGL,Alphabet Inc Class A,Information Technology,2.206512,1.9
5,GOOG,Alphabet Inc Class C,Information Technology,2.043843,1.9
6,AMZN,Amazon.com Inc,Consumer Discretionary,3.784923,1.7
7,MSFT,Microsoft Corp.,Information Technology,5.978413,1.7
8,ACN,Accenture plc,Information Technology,0.542795,1.6
9,ADS,Alliance Data Systems,Information Technology,NaN,1.6


In [46]:

Apple_df = test("AAPL", merged_df)
Apple_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Score
0,AAPL,Apple Inc.,Information Technology,0.0
1,MSFT,Microsoft Corp.,Information Technology,2.8
2,ACN,Accenture plc,Information Technology,2.5
3,ADS,Alliance Data Systems,Information Technology,2.5
4,ADP,Automatic Data Processing,Information Technology,2.5
5,FB,"Facebook, Inc.",Information Technology,2.5
6,HRS,Harris Corporation,Information Technology,2.5
7,KLAC,KLA-Tencor Corp.,Information Technology,2.5
8,LRCX,Lam Research,Information Technology,2.5
9,SWKS,Skyworks Solutions,Information Technology,2.5


We also provided a test case for a low market cap stock and low weighted stock like Ralph Lauren. As you can see, the stocks that are recommended to it are other low market cap and low weighted stocks. This shows that are algorithm is working since we wanted to place greater emphasis on market cap and weight. Comparing this to our original functions output for Ralph Lauren, we can again see that there are recommendations from new sectors, not just the Consumer Discretionary sector.


Overall, this goes to show that there are many different approaches to a recommendation system for stocks. The best recommendation system really depends on what the user is looking for. If a user wants stocks that are generally from the same sector with similar dividend yield then our orginal recommender system would be great for them. On the other hand, if a user wants stocks that are similar based on weight in the S&P 500 index and similar with regards to market cap, then they will choose to use our new recommender system. This just goes to show that recommednations can be made on various different categories so it can really depend on what type of recommendation someone is looking for.

Though, we do believe that our original algorithm worked very well because it was able to nicely combine weight, dividend yield, and sector in its recommendations.

In [48]:
RL_df_alt = test_new("RL", merged_df)
RL_df_alt

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Weight,Score
0,RL,Polo Ralph Lauren Corp.,Consumer Discretionary,0.014877,0.0
1,BWA,BorgWarner,Consumer Discretionary,0.024189,3.1
2,AOS,A.O. Smith Corp,Industrials,0.023597,2.9
3,DISCA,Discovery Communications-A,Consumer Discretionary,0.011455,2.9
4,DISCK,Discovery Communications-C,Consumer Discretionary,0.021476,2.9
5,DISH,Dish Network,Consumer Discretionary,0.020584,2.9
6,BEN,Franklin Resources,Financials,0.020173,2.9
7,NWL,Newell Brands,Consumer Discretionary,0.022112,2.9
8,NCLH,Norwegian Cruise Line,Consumer Discretionary,0.021769,2.9
9,NRG,NRG Energy,Utilities,0.024194,2.9


In [49]:
RL_df = test("RL", merged_df)
RL_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Company Name,Sector,Score
0,RL,Polo Ralph Lauren Corp.,Consumer Discretionary,0.0
1,BWA,BorgWarner,Consumer Discretionary,3.8
2,NWSA,News Corp. Class A,Consumer Discretionary,3.4
3,NWS,News Corp. Class B,Consumer Discretionary,3.4
4,DISCA,Discovery Communications-A,Consumer Discretionary,3.1
5,DISCK,Discovery Communications-C,Consumer Discretionary,3.1
6,DISH,Dish Network,Consumer Discretionary,3.1
7,NWL,Newell Brands,Consumer Discretionary,3.1
8,NCLH,Norwegian Cruise Line,Consumer Discretionary,3.1
9,FOX,Twenty-First Century Fox Class B,Consumer Discretionary,3.1
